### Predict items to buy

Need to use (user, sku) pairs for the prediction

Use simple analysis to look at the probability


In [10]:
import base
import numpy as np
import pandas as pd

pus = base.ProductUserSet(product_cate=8)

In [11]:
data = pd.DataFrame(pus.get_user_sku_action('favor', pus.TYPE_FAVOR, 0, 6e6))
data = data.join(pus.get_user_sku_action('buy1', pus.TYPE_BUY, 0, 3e6), how='outer')
data = data.join(pus.get_user_sku_action('buy2', pus.TYPE_BUY, 3e6, 5e6), how='outer')
data = data.join(pus.get_user_sku_action('buy3', pus.TYPE_BUY, 5e6, 6e6), how='outer')
data = data.join(pus.get_user_sku_action('view1', pus.TYPE_VIEW, 0, 3e6), how='outer')
data = data.join(pus.get_user_sku_action('view2', pus.TYPE_VIEW, 3e6, 5e6), how='outer')
data = data.join(pus.get_user_sku_action('view3', pus.TYPE_VIEW, 5e6, 6e6), how='outer')
data = data.join(pus.get_user_sku_cart('cart1', 0, 3e6, True), how='outer')
data = data.join(pus.get_user_sku_cart('cart2', 0, 6e6, True), how='outer')
data = data.join(pus.get_user_sku_action('click1', pus.TYPE_CLICK, 0, 3e6), how='outer')
data = data.join(pus.get_user_sku_action('click2', pus.TYPE_CLICK, 3e6, 5e6), how='outer')
data = data.join(pus.get_user_sku_action('click3', pus.TYPE_CLICK, 5e6, 6e6), how='outer')
data = data.join(pus.get_user_sku_action('final', pus.TYPE_BUY, 6e6, 6e6 + 5 * 24 *3600), how='outer')
data[pd.isnull(data)] = 0

In [12]:
def factor_analyse(col1, col2, dt=data, print_out=True):
    f1 = dt[col1] > 0
    f2 = dt[col2] > 0
    num1 = np.sum(np.logical_and(f1, f2))
    num2 = np.sum(np.logical_and(f1, np.logical_not(f2)))
    num3 = np.sum(np.logical_and(np.logical_not(f1), f2))
    num4 = np.sum(np.logical_not(np.logical_or(f1, f2)))
    if print_out:
        print '\t{}\t{}'.format(col2, '-')
        print '{}\t{}\t{}'.format(col1, num1, num2)
        print '{}\t{}\t{}'.format('-', num3, num4)
    return num1, num2, num3, num4

In [13]:
buy_stat = pd.DataFrame()
for col in data.columns:
    buy_stat[col] = factor_analyse(col, 'final')

import pickle
with open('data/probability', 'w') as f:
    pickle.dump(buy_stat, f)

	final	-
favor	88	34513
-	1226	1293045
	final	-
buy1	0	3229
-	1314	1324329
	final	-
buy2	0	5553
-	1314	1322005
	final	-
buy3	0	2850
-	1314	1324708
	final	-
view1	81	446523
-	1233	881035
	final	-
view2	209	641314
-	1105	686244
	final	-
view3	482	316531
-	832	1011027
	final	-
cart1	18	29841
-	1296	1297717
	final	-
cart2	316	84195
-	998	1243363
	final	-
click1	73	437872
-	1241	889686
	final	-
click2	204	611369
-	1110	716189
	final	-
click3	458	303699
-	856	1023859
	final	-
final	1314	0
-	0	1327558


In [14]:
for col in buy_stat.columns:
    prob1 = buy_stat[col][0] * 1.0 / (buy_stat[col][0] + buy_stat[col][1])
    prob2 = buy_stat[col][2] * 1.0 / (buy_stat[col][2] + buy_stat[col][3])
    if prob2 == 0.0:
        prob2 += 1e-8
    print col, '\t+\t', prob1, '\t-\t', prob2, '\tratio\t', prob1 / prob2

favor 	+	0.00254327909598 	-	0.000947251387074 	ratio	2.68490406104
buy1 	+	0.0 	-	0.000991217092385 	ratio	0.0
buy2 	+	0.0 	-	0.000992957858234 	ratio	0.0
buy3 	+	0.0 	-	0.000990933785412 	ratio	0.0
view1 	+	0.000181368729344 	-	0.00139753453599 	ratio	0.12977763674
view2 	+	0.000325787228205 	-	0.00160762582036 	ratio	0.202651154317
view3 	+	0.00152044237933 	-	0.000822248949705 	ratio	1.84912656912
cart1 	+	0.000602833316588 	-	0.000997680546692 	ratio	0.604234811019
cart2 	+	0.00373915821609 	-	0.00080201806389 	ratio	4.66218703099
click1 	+	0.000166687597758 	-	0.00139293118291 	ratio	0.119666785985
click2 	+	0.00033356606652 	-	0.00154747183532 	ratio	0.215555500855
click3 	+	0.00150580128026 	-	0.000835354220442 	ratio	1.8025901389
final 	+	1.0 	-	1e-08 	ratio	100000000.0


In [15]:
# More complex interactions
for col in data.columns:
    if col != 'final':
        sub_set = data[data[col] > 0]
        print col, "+ subset"

        for col2 in sub_set.columns:
            if col2 != col and col2 != 'final':
                ret = factor_analyse(col2, 'final', sub_set, False)
                prob1 = ret[0] * 1.0 / (ret[0] + ret[1])
                prob2 = ret[2] * 1.0 / (ret[2] + ret[3])
                if ret[2] == 0:
                    prob2 = 1.0 / (ret[2] + ret[3])
                print col2, '\t', prob1, '\t', prob2, '\t', prob1 / prob2
    print

favor + subset
buy1 	0.0 	0.00256672014 	0.0
buy2 	0.0 	0.00258953005915 	0.0
buy3 	0.0 	0.00256769374416 	0.0
view1 	0.000952450728991 	0.00357961053837 	0.266076635651
view2 	0.00182978092082 	0.00354659248957 	0.515926463558
view3 	0.00642292490119 	0.000445295453533 	14.4239624506
cart1 	0.00171673819742 	0.00260295621456 	0.659534028204
cart2 	0.00661157024793 	0.00157446503972 	4.1992486852
click1 	0.000750018750469 	0.00366748166259 	0.204505112628
click2 	0.0018812283913 	0.00341525480479 	0.550831050338
click3 	0.00649405414523 	0.000483644037988 	13.4273424981

buy1 + subset
favor 	0.0 	0.000343288705802 	0.0
buy2 	0.0 	0.000309789343247 	0.0
buy3 	nan 	0.000309693403531 	nan
view1 	0.0 	0.0277777777778 	0.0
view2 	0.0 	0.000503524672709 	0.0
view3 	0.0 	0.000346380325598 	0.0
cart1 	0.0 	0.00170648464164 	0.0
cart2 	0.0 	0.00171821305842 	0.0
click1 	0.0 	0.00621118012422 	0.0
click2 	0.0 	0.000489476260401 	0.0
click3 	0.0 	0.000344708721131 	0.0

buy2 + subset
favor 	0.0 	

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


favor 	0.000952450728991 	0.000157060202561 	6.06423978486
buy1 	0.0 	0.000182674764496 	0.0
buy2 	0.0 	0.000181859003143 	0.0
buy3 	0.0 	0.000181467060368 	0.0
view2 	0.000643259981749 	0.000100063988287 	6.42848633919
view3 	0.00209674273454 	8.91817806316e-05 	23.5108866373
cart1 	0.000592724309106 	0.000154905745812 	3.82635457452
cart2 	0.0013228894691 	0.000102900600413 	12.8559936851
click1 	0.000171846258108 	0.000299007295778 	0.574722625617
click2 	0.000665089013658 	9.94892014944e-05 	6.68503720672
click3 	0.00206019798 	9.37420172813e-05 	21.9773164665

view2 + subset
favor 	0.00182978092082 	0.000276837995049 	6.60957293994
buy1 	0.0 	0.000326419691385 	0.0
buy2 	0.0 	0.0003286034354 	0.0
buy3 	0.0 	0.000326184330042 	0.0
view1 	0.000643259981749 	0.000288858417613 	2.22690405585
view3 	0.00198789028305 	0.000170447475758 	11.6627733805
cart1 	0.00109569028488 	0.00031241525538 	3.50715999303
cart2 	0.0022734927585 	0.00017002823142 	13.3712662863
click1 	0.000662976610802

#### Summary

Favor will increase probability for buying.

Recent click/viewing/adding to cart increases probability.

If the item is added to cart/viewed/clicked early on, it's less likely to be bought.

If the item has been bought, it will never be bought again.


In [16]:
train_data = data.drop('final', 1).as_matrix()
train_label = np.array(data['final'] > 0, dtype=int)
mean_dt = np.mean(train_data, axis=0)
std_dt = np.std(train_data, axis=0)
train_data = (train_data - mean_dt) / std_dt

In [17]:
nrow = len(train_data)
test_size = int(nrow * 0.3)
import random
idx = random.sample(xrange(nrow), test_size)
mask = np.ones(nrow, dtype=bool)
mask[idx] = False
np.save('train_data4', train_data[mask, :])
np.save('train_label4', train_label[mask])
np.save('test_data4', train_data[idx, :])
np.save('test_label4', train_label[idx])